In [1]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline
import torch

from torch.autograd import Variable
from torch.utils.data import Dataset

import h5py
import numpy as np
import os

from tabulate import tabulate
import sep_eval 
import beamformers

from scipy.io import wavfile

import IPython.display as ipd

In [2]:
class WSJDataset(Dataset):
    """
    Wrapper for the WSJ Dataset.
    """
    
    def __init__(self, path):
        super(WSJDataset, self).__init__()

        self.h5pyLoader = h5py.File(path, 'r')
        
        self.ane_source = self.h5pyLoader['anechoic_source']
        self.e_source = self.h5pyLoader['echoic_source']
        self.ane_noise = self.h5pyLoader['anechoic_noise']
        self.e_noise = self.h5pyLoader['echoic_noise']
        self.mask = self.h5pyLoader['mask']
        
        self._len = self.mask.shape[0]
    
    def __getitem__(self, index):
        # mic, source, t
        ane_source_tensor = self.ane_source[index].astype(np.float32)
        e_source_tensor = self.e_source[index].astype(np.float32)
        ane_noise_tensor = self.ane_noise[index].astype(np.float32)
        e_noise_tensor = self.e_noise[index].astype(np.float32)
        
        return ane_source_tensor, e_source_tensor, ane_noise_tensor, e_noise_tensor
    
    def __len__(self):
        return self._len

In [5]:
cv = WSJDataset('/Data/DATASETS/WSJ/cv')
print len(cv)
ex = cv[40]
for e in ex:
    print e.shape

IOError: Unable to open file (unable to open file: name = '/Data/DATASETS/WSJ/cv', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [12]:
# Oracle MVDR
# Oracle MWF
# Beamformit 
# SDW_MWF
# MWF
reload(beamformers)
fs = 8000
receptive_field = 0.064  # in s
frame_len = int(fs * receptive_field)
frame_step = int(frame_len / 4)
n_ch = 4
spk = ex[1][:n_ch, 0]
spk /= np.max(np.abs(spk))

# talker
nn = ex[1][:n_ch, 1]
nn /= np.max(np.abs(nn))

gt = ex[1][0, 0]
mix = spk + nn
mix /= np.max(np.abs(mix))

eval_before = sep_eval.full_eval(mix[0], gt, avg=True)

# SDW_MWF - A
print("SDW_MWF - With reference")
out_sdwmwf_a = beamformers.SDW_MWF(mix, nn, spk, frame_len=frame_len, frame_step=frame_step)
sdwmwf_after_a = sep_eval.full_eval(out_sdwmwf_a, gt, avg=True)

# SDW_MWF - B
print("SDW_MWF - Without reference")
out_sdwmwf_b = beamformers.SDW_MWF(mix, nn, reference=None, frame_len=frame_len, frame_step=frame_step)
sdwmwf_after_b = sep_eval.full_eval(out_sdwmwf_b, gt, avg=True)

# MWF
print("Oracle MWF")
out_mwf = beamformers.MWF_Oracle(mix , spk, nn, frame_len=frame_len, frame_step=frame_step)
mwf_after = sep_eval.full_eval(out_mwf, gt, avg=True)

# MVDR - A
print("MVDR - With reference")
out_mvdr_a = beamformers.MVDR(mix, nn, spk, frame_len=frame_len, frame_step=frame_step)
mvdr_after_a = sep_eval.full_eval(out_mvdr_a, gt, avg=True)

# MVDR - B
print("MVDR - Without reference")
out_mvdr_b = beamformers.MVDR(mix, nn, reference=None, frame_len=frame_len, frame_step=frame_step)
mvdr_after_b = sep_eval.full_eval(out_mvdr_b, gt, avg=True)

# MSNR - A
print("MSNR - Without reference")
out_msnr_a = beamformers.MSNR(mix, nn, spk, frame_len=frame_len, frame_step=frame_step)
msnr_after_a = sep_eval.full_eval(out_msnr_a, gt, avg=True)

# MSNR - B
print("MSNR - Without reference")
out_msnr_b = beamformers.MSNR(mix, nn, reference=None, frame_len=frame_len, frame_step=frame_step)
msnr_after_b = sep_eval.full_eval(out_msnr_b, gt, avg=True)

# TD-MVDR - A
print("TD_MVDR - With reference")
out_tdmvdr_a = beamformers.TD_MVDR(mix, nn, spk, frame_len=frame_len)
tdmvdr_after_a = sep_eval.full_eval(out_tdmvdr_a, gt, avg=True)

# TD-MVDR - B
print("TD_MVDR - Without reference")
out_tdmvdr_b = beamformers.TD_MVDR(mix, nn, reference=None, frame_len=frame_len)
tdmvdr_after_b = sep_eval.full_eval(out_tdmvdr_b, gt, avg=True)

# TD-MWF - A
print("TD_MWF - With reference")
out_tdmwf_a = beamformers.TD_MWF(mix, nn, spk, frame_len=frame_len)
tdmwf_after_a = sep_eval.full_eval(out_tdmwf_a, gt, avg=True)

# TD-MWF - B
print("TD_MWF - Without reference")
out_tdmwf_b = beamformers.TD_MWF(mix, nn, reference=None, frame_len=frame_len)
tdmwf_after_b = sep_eval.full_eval(out_tdmwf_b, gt, avg=True)

# BeamformIt
print("Beamformit")
out_bfi = beamformers.BeamformIt(mix)
bfi_after = sep_eval.full_eval(out_bfi, gt, avg=True)

# table
table = [["SDR", 
          eval_before['sdr'], 
          mwf_after['sdr'], 
          sdwmwf_after_a['sdr'],
          sdwmwf_after_b['sdr'],
          mvdr_after_a['sdr'], 
          mvdr_after_b['sdr'], 
          msnr_after_a['sdr'], 
          msnr_after_b['sdr'], 
          tdmvdr_after_a['sdr'], 
          tdmvdr_after_b['sdr'], 
          tdmwf_after_a['sdr'], 
          tdmwf_after_b['sdr'], 
          bfi_after['sdr'],
         ],
         ["PESQ", 
          eval_before['pesq'], 
          mwf_after['pesq'], 
          sdwmwf_after_a['pesq'],
          sdwmwf_after_b['pesq'],
          mvdr_after_a['pesq'], 
          mvdr_after_b['pesq'], 
          msnr_after_a['pesq'], 
          msnr_after_b['pesq'], 
          tdmvdr_after_a['pesq'], 
          tdmvdr_after_b['pesq'], 
          tdmwf_after_a['pesq'], 
          tdmwf_after_b['pesq'], 
          bfi_after['pesq'],
         ], 
         ["STOI", 
          eval_before['stoi'], 
          mwf_after['stoi'], 
          sdwmwf_after_a['stoi'],
          sdwmwf_after_b['stoi'],
          mvdr_after_a['stoi'], 
          mvdr_after_b['stoi'], 
          msnr_after_a['stoi'], 
          msnr_after_b['stoi'], 
          tdmvdr_after_a['stoi'], 
          tdmvdr_after_b['stoi'], 
          tdmwf_after_a['stoi'], 
          tdmwf_after_b['stoi'], 
          bfi_after['stoi'],
         ]]

head = ["Measure", 
        "Before", 
        "MWF", 
        "SMWF-A", 
        "SMWF-B",  
        "MVDR-A", 
        "MVDR-B",
        "MSNR-A", 
        "MSNR-B",
        "TDMVDR-A", 
        "TDMVDR-B",
        "TDMWF-A", 
        "TDMWF-B",
        "BFiT"]

print(tabulate(table, headers=head, tablefmt='fancy_grid'))

ipd.display(ipd.Audio(mix[0], rate=fs))
ipd.display(ipd.Audio(out_mwf, rate=fs))
ipd.display(ipd.Audio(out_sdwmwf_a, rate=fs))
ipd.display(ipd.Audio(out_sdwmwf_b, rate=fs))
ipd.display(ipd.Audio(out_mvdr_a, rate=fs))
ipd.display(ipd.Audio(out_mvdr_b, rate=fs))
ipd.display(ipd.Audio(out_msnr_a, rate=fs))
ipd.display(ipd.Audio(out_msnr_b, rate=fs))
ipd.display(ipd.Audio(out_tdmvdr_a, rate=fs))
ipd.display(ipd.Audio(out_tdmvdr_b, rate=fs))
ipd.display(ipd.Audio(out_tdmwf_a, rate=fs))
ipd.display(ipd.Audio(out_tdmwf_b, rate=fs))
ipd.display(ipd.Audio(out_bfi, rate=fs))
ipd.display(ipd.Audio(gt, rate=fs))

SDW_MWF - With reference
SDW_MWF - Without reference
Oracle MWF
MVDR - With reference
MVDR - Without reference
MSNR - Without reference
MSNR - Without reference
TD_MVDR - With reference
TD_MVDR - Without reference
TD_MWF - With reference
TD_MWF - Without reference
Beamformit


/home/enea/anaconda2/lib/python2.7/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


╒═══════════╤══════════╤══════════════╤══════════╤══════════╤══════════╤══════════╤══════════╤══════════╤════════════╤════════════╤═══════════╤═══════════╤═══════════╕
│ Measure   │   Before │          MWF │   SMWF-A │   SMWF-B │   MVDR-A │   MVDR-B │   MSNR-A │   MSNR-B │   TDMVDR-A │   TDMVDR-B │   TDMWF-A │   TDMWF-B │      BFiT │
╞═══════════╪══════════╪══════════════╪══════════╪══════════╪══════════╪══════════╪══════════╪══════════╪════════════╪════════════╪═══════════╪═══════════╪═══════════╡
│ SDR       │ 2.82732  │ -29.2122     │ 4.36812  │ 4.20877  │ 4.66049  │  4.53317 │ 4.36812  │ 4.20877  │   4.98109  │   3.12328  │ -1.03138  │ -0.989396 │ -1.06211  │
├───────────┼──────────┼──────────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼────────────┼────────────┼───────────┼───────────┼───────────┤
│ PESQ      │ 2.202    │   0.963      │ 2.69     │ 2.672    │ 2.566    │  2.567   │ 2.69     │ 2.672    │   2.715    │   2.298    │  2.255    │  2.278    │  1.9

In [21]:
import soundfile as sf
sf.write('wavs/mix.wav', mix.T, 8000)
sf.write('wavs/nn.wav', nn.T, 8000)
sf.write('wavs/spk.wav', spk.T, 8000)
sf.write('wavs/gt.wav', gt, 8000)

In [ ]:
from tqdm import tqdm_notebook as tqdm

frame_len = 512
frame_step = 1
M = 4
ref = 1
n_win = (spk.shape[1] - frame_len) // frame_step

R_y = 0.0
R_in = 0.0
rho_yY = 0.0
rho_vV = 0.0
rho_xX = 0.0

mix = spk + nn

var_y = np.var(mix[ref])
var_v = np.var(nn[ref])
var_x = np.var(spk[ref])

for i in tqdm(range(n_win)):
    j = i * frame_step
    v = nn[:, j:j + frame_len]  # interf
    y = mix[:, j:j + frame_len]  # mics output
    x = spk[:, j:j + frame_len]
    
    Y = y.reshape(-1)
    V = v.reshape(-1)
    X = x.reshape(-1)
    
    R_y += np.outer(Y, Y)
    R_in += np.outer(V, V)
    rho_yY += y[ref][-1] * Y 
    rho_vV += v[ref][-1] * V 
    rho_xX += x[ref][-1] * X 
    

In [ ]:
A = np.outer(V1, V1)
B = np.outer(V2, V2)

In [ ]:
n_win = 1000

In [ ]:
import gc
gc.disable()
R = 0.0
for i in tqdm(range(n_win)):
    y = mix[:, i:i + frame_len].T.reshape(-1)
    R += np.outer(y, y)
    
# R = sum(R)

In [ ]:
ch = 4
V1 = mix[:, :frame_len].T.reshape(-1)
R1 = np.outer(V1, V1)
prev_Q = np.zeros_like(R1)
_Q = np.zeros_like(R1)
prev_Q += R1

for i in tqdm(range(1, n_win)):
    _b = mix[:, i:i + frame_len].T.reshape(-1)
    _Q = np.zeros_like(R1)
    _Q[:-ch, :-ch] = prev_Q[ch:, ch:]
    _c = np.outer(_b[-ch:], _b)
    _Q[-ch:, :] = _c
    _Q[:-ch, -ch:] = _c[:, :-ch].T
#     for k in range(1, ch + 1):
#         _a = _b[-k] * _b
#         _Q[-k, :] = _a
#         _Q[:-ch, -k] = _a[:-ch]
    prev_Q = np.zeros_like(R1)
    prev_Q = _Q
    R1 += _Q



In [ ]:
print np.mean(R - R1)

In [ ]:
R = np.zeros_like(R1)
for i in tqdm(range(n_win)):
    y = mix[:, i:i + frame_len].T.reshape(-1)
    R += np.outer(y, y)
    
V1 = mix[:, 0:0 + frame_len].T.reshape(-1)
V2 = mix[:, 1:1 + frame_len].T.reshape(-1)
V3 = mix[:, 2:2 + frame_len].T.reshape(-1)
V4 = mix[:, 3:3 + frame_len].T.reshape(-1)

R1 = np.outer(V1, V1) + np.outer(V2, V2) + np.outer(V3, V3) + np.outer(V4, V4)

print np.mean(R - R1)

In [ ]:
ch = 4
V1 = mix[:, 0:0 + frame_len].T.reshape(-1)
V2 = mix[:, 1:1 + frame_len].T.reshape(-1)
V3 = mix[:, 2:2 + frame_len].T.reshape(-1)
V4 = mix[:, 3:3 + frame_len].T.reshape(-1)

R1 = np.outer(V1, V1) + np.outer(V2, V2) + np.outer(V3, V3) + np.outer(V4, V4)


# V1
Q1 = np.outer(V1, V1)
prev_Q = np.zeros_like(Q1)
prev_Q += Q1

# V4  
for i in range(1, 4):
    V4 = mix[:, i:i + frame_len].T.reshape(-1)
    _Q = np.zeros_like(Q1)
    _Q[:-ch, :-ch] = prev_Q[ch:, ch:]
    for k in range(1, ch + 1):
        _a = V4[-k] * V4
        _Q[-k, :] = _a
        _Q[:-ch, -k] = _a[:-ch]
    Q1 += _Q  
    prev_Q = np.zeros_like(Q1)
    prev_Q += _Q

print np.mean(Q1 - R1)
print np.mean(R1 - R)


In [ ]:
ch = 4
V1 = mix[:, :frame_len].T.reshape(-1)
V2 = mix[:, 1:1 + frame_len].T.reshape(-1)
V3 = mix[:, 2:2 + frame_len].T.reshape(-1)
V4 = mix[:, 3:3 + frame_len].T.reshape(-1)

R1 = np.outer(V1, V1) + np.outer(V2, V2) + np.outer(V3, V3) + np.outer(V4, V4)

d = M * frame_len

# V1
Q1 = np.outer(V1, V1)
prev_Q = np.zeros_like(Q1)
prev_Q += Q1


# V2
_Q = np.zeros_like(Q1)
_Q[:-ch, :-ch] = prev_Q[ch:, ch:]
for k in range(1, ch + 1):
    _a = V2[-k] * V2
    _Q[-k, :] = _a
    _Q[:-ch, -k] = _a[:-ch]
Q1 += _Q   

prev_Q = np.zeros_like(Q1)
prev_Q += _Q


# V3    
_Q = np.zeros_like(Q1)
_Q[:-ch, :-ch] = prev_Q[ch:, ch:]
for k in range(1, ch + 1):
    _a = V3[-k] * V3
    _Q[-k, :] = _a
    _Q[:-ch, -k] = _a[:-ch]
Q1 += _Q  

prev_Q = np.zeros_like(Q1)
prev_Q += _Q


# V4  
_Q = np.zeros_like(Q1)
_Q[:-ch, :-ch] = prev_Q[ch:, ch:]
for k in range(1, ch + 1):
    _a = V4[-k] * V4
    _Q[-k, :] = _a
    _Q[:-ch, -k] = _a[:-ch]
Q1 += _Q  

print np.mean(Q1 - R1)

# for i in tqdm(range(1, n_win)):
#     _V = mix[:, i:i + frame_len].reshape(-1)
#     R_y_2[:-1, :-1] +=  R_y_2[1:, 1:] 
#     _p = _V * _V[-1]
#     R_y_2[-1, :] += _p
#     R_y_2[:-1, -1] += _p[:-1]
# print plt.imshow(R_y[:10, :10], aspect='auto')

In [ ]:
# calculate weights with collected statistics
rho_xX = (rho_yY - rho_vV) / (var_y - var_v) / n_win
# rho_xX2 = rho_xX / var_x / n_win

# R_in - inv
part = np.linalg.inv(R_y / n_win + np.eye(M * frame_len) * 1e-15).dot(rho_xX) # / np.linalg.norm(rho_xX)
h_MVDR = part / rho_xX.T.dot(part) 
h_MVDR /= np.sqrt(np.sum(h_MVDR ** 2))

part = np.linalg.inv(R_in / n_win + np.eye(M * frame_len) * 1e-15).dot(rho_xX) # / np.linalg.norm(rho_xX)
h_MWF = (part * var_x) / (1 + var_x * rho_xX.T.dot(part))
h_MWF /= np.sqrt(np.sum(h_MWF ** 2))

y_MVDR = np.zeros((spk.shape[1],))
y_MWF  = np.zeros((spk.shape[1],))

for i in tqdm(range(spk.shape[1] - frame_len)):
    y = mix[:, i:i + frame_len].reshape(-1)
    y_MVDR[frame_len + i] = h_MVDR.dot(y)
    y_MWF[frame_len + i] = h_MWF.dot(y)

y_MVDR /= np.max(np.abs(y_MVDR))
y_MWF /= np.max(np.abs(y_MWF))

eval_before = sep_eval.full_eval(mix[0], gt, avg=True)
mvdr_after = sep_eval.full_eval(y_MVDR, gt, avg=True)
mwf_after = sep_eval.full_eval(y_MWF, gt, avg=True)
    
table = [["SDR",eval_before['sdr'], mvdr_after['sdr'], mwf_after['sdr']],
         ["PESQ",eval_before['pesq'], mvdr_after['pesq'], mwf_after['pesq']], 
         ["STOI",eval_before['stoi'], mvdr_after['stoi'], mwf_after['stoi']]]

head = ["Measure", "Before", "MVDR", "MWF"]
print tabulate(table, headers=head, tablefmt='fancy_grid')

ipd.display(ipd.Audio(mix[0], rate=8000))
ipd.display(ipd.Audio(y_MVDR, rate=8000))
ipd.display(ipd.Audio(y_MWF, rate=8000))
ipd.display(ipd.Audio(gt, rate=8000))

In [ ]:
plt.plot(gt/np.max(np.abs(gt)))
plt.plot(y_MVDR/np.max(np.abs(y_MVDR)))

k = np.correlate(y_MVDR, gt, 'full')
plt.figure()
plt.plot(k)
print np.argmax(k) - len(gt)

In [ ]:
plt.plot(h_MVDR_in_inv - h_MWF_in_pinv)
# plt.plot(h_MVDR_in_pinv)
# plt.plot(h_MWF_in_pinv)

In [ ]:
full_sig = []
rho_xX = rho_xX / n_win
part = np.linalg.pinv(R_in / n_win + np.eye(M * frame_len)*1e-15).dot(rho_xX)
h_MVDR = part / rho_xX.T.dot(part)
h_MWF = (part * var_x) / (1 + var_x * rho_xX.T.dot(part))


for i in tqdm(range(spk.shape[1] - frame_len)):
    y = mix[:, i:i + frame_len].reshape(-1)
    full_sig.append(h_MVDR.dot(y))
full_sig.extend([0] * frame_len)
    
out_tdmvdr = np.array(full_sig)
# out_tdmvdr /= np.max(np.abs(out_tdmvdr))
tdmvdr_after = sep_eval.full_eval(out_tdmvdr, gt, avg=True)
print tdmvdr_after
ipd.display(ipd.Audio(mix[0] / np.max(np.abs(mix[0])), rate=8000))
ipd.display(ipd.Audio(gt / np.max(np.abs(gt)), rate=8000))
ipd.display(ipd.Audio(out_tdmvdr / np.max(np.abs(out_tdmvdr)), rate=8000))


In [ ]:
plt.plot(out_tdmvdr)

In [ ]:
from tqdm import tqdm_notebook as tqdm
def rho(v, m):
    return np.correlate(v, m.reshape(-1), 'same')
#     return np.hstack([np.correlate(v, _m, 'same') for _m in m])

frame_len = 512
frame_step = 8
M = 4
ref = 1
n_win = (spk.shape[1] - frame_len) // frame_step
full_sig = []
R_in = np.zeros((frame_len, M, M))
rho_yY = 0.0
rho_vV = 0.0
rho_xX = 0.0

mix = spk + nn

var_y = np.var(mix[ref])
var_v = np.var(nn[ref])

for i in tqdm(range(n_win)):
    j = i * frame_step
    v = nn[:, j:j + frame_len]  # interf
    y = mix[:, j:j + frame_len]  # mics output
    x = spk[:, j:j + frame_len]  
    
    Y = y.reshape(-1)
    V = v.reshape(-1)
    X = x.reshape(-1)
    
    for k in range(frame_len):
        R_in[k] += np.outer(v[:, k], v[:, k])
        
    rho_yY += y[ref][-1] * Y 
    rho_vV += v[ref][-1] * V
    rho_xX += x[ref][-1] * X

# calculate weights with collected statistics
# rho_xX = (rho_yY - rho_vV) # / (var_y - var_v) / n_win
rho_xX = rho_xX / n_win

rho_xX = rho_xX.reshape(M, frame_len)
h_MVDR = np.zeros((M, frame_len))
for k in range(frame_len):
    part = np.linalg.inv(R_in[k] / n_win).dot(rho_xX[:, k])
    h_MVDR[:, k] = part / rho_xX[:, k].dot(part)

h_MVDR = h_MVDR.reshape(-1)   
for i in tqdm(range(spk.shape[1] - frame_len)):
    y = mix[:, i:i + frame_len].reshape(-1)
    full_sig.append(h_MVDR.dot(y))
full_sig.extend([0] * frame_len)
    
out_tdmvdr = np.array(full_sig)
tdmvdr_after = sep_eval.full_eval(out_tdmvdr, gt, avg=True)
print tdmvdr_after
ipd.display(ipd.Audio(out_tdmvdr / np.max(np.abs(out_tdmvdr)), rate=8000))

In [7]:
# Oracle MVDR
# Oracle MWF
# Beamformit 
# SDW_MWF
# MWF
fs = 8000
receptive_field = 0.128  # in s
frame_len = int(fs * receptive_field)
frame_step = int(frame_len / 4)
n_ch = 4
spk = ex[1][:n_ch, 0]
spk /= np.max(np.abs(spk))

# talker
nn = ex[1][:n_ch, 1]
nn /= np.max(np.abs(nn))

# noise
# nn = ex[2][:n_ch, 0]
# nn /= np.max(np.abs(nn))

gt = ex[0][0, 0]
mix = spk + nn
mix /= np.max(np.abs(mix))

eval_before = sep_eval.full_eval(mix[0], gt, avg=True)

# SDW_MWF
out_sdwmwf = beamformers.SDW_MWF(mix, spk, nn, frame_len=frame_len, frame_step=frame_step)
sdwmwf_after = sep_eval.full_eval(out_sdwmwf, gt, avg=True)

# MVDR
out_mvdr = beamformers.MVDR(mix, spk, nn, frame_len=frame_len, frame_step=frame_step)
mvdr_after = sep_eval.full_eval(out_mvdr, gt, avg=True)

# MVDRb
out_mvdrb = beamformers.MVDR(mix, None, nn, frame_len=frame_len, frame_step=frame_step)
mvdr_afterb = sep_eval.full_eval(out_mvdrb, gt, avg=True)

# MSNR
out_msnr = beamformers.MSNR(mix, spk, nn, frame_len=frame_len, frame_step=frame_step)
msnr_after = sep_eval.full_eval(out_msnr, gt, avg=True)

# table
table = [["SDR",eval_before['sdr'], sdwmwf_after['sdr'], mvdr_after['sdr'], mvdr_afterb['sdr'], msnr_after['sdr']],
         ["PESQ",eval_before['pesq'], sdwmwf_after['pesq'], mvdr_after['pesq'], mvdr_afterb['pesq'], msnr_after['pesq']], 
         ["STOI",eval_before['stoi'], sdwmwf_after['stoi'], mvdr_after['stoi'], mvdr_afterb['stoi'], msnr_after['stoi']]]

head = ["Measure", "Before", "SDW_MWF", "MVDR", "MVDRb", "MSNR"]
print tabulate(table, headers=head, tablefmt='fancy_grid')

ipd.display(ipd.Audio(mix[0], rate=fs))
ipd.display(ipd.Audio(out_sdwmwf, rate=fs))
ipd.display(ipd.Audio(out_mvdr, rate=fs))
ipd.display(ipd.Audio(out_msnr, rate=fs))
ipd.display(ipd.Audio(gt, rate=fs))

╒═══════════╤═══════════╤═════════════╤═════════════╤═════════════╤═════════════╕
│ Measure   │    Before │     SDW_MWF │        MVDR │       MVDRb │        MSNR │
╞═══════════╪═══════════╪═════════════╪═════════════╪═════════════╪═════════════╡
│ SDR       │ -1.44539  │ -49.6966    │ -35.3339    │ -35.3339    │ -49.6963    │
├───────────┼───────────┼─────────────┼─────────────┼─────────────┼─────────────┤
│ PESQ      │  1.747    │   0.546     │   0.631     │   0.631     │   0.546     │
├───────────┼───────────┼─────────────┼─────────────┼─────────────┼─────────────┤
│ STOI      │  0.508417 │   0.0309265 │   0.0807818 │   0.0807818 │   0.0309265 │
╘═══════════╧═══════════╧═════════════╧═════════════╧═════════════╧═════════════╛


In [9]:
# Oracle MVDR
# Oracle MWF
# Beamformit 
# SDW_MWF
# MWF
reload(beamformers)
fs = 8000
receptive_field = 0.128  # in s
frame_len = int(fs * receptive_field)
frame_step = int(frame_len / 4)
n_ch = 4
spk = ex[1][:n_ch, 0]
spk /= np.max(np.abs(spk))

# talker
nn = ex[1][:n_ch, 1]
nn /= np.max(np.abs(nn))

# noise
# nn = ex[2][:n_ch, 0]
# nn /= np.max(np.abs(nn))

gt = ex[1][0, 0]
gt2 = ex[1][0, 1]
mix = spk + nn
mix /= np.max(np.abs(mix))

eval_before = sep_eval.full_eval(mix[0], gt, avg=True)


# MVDR
out_mvdr = beamformers.MVDR(mix, nn, spk, frame_len=frame_len, frame_step=frame_step)
mvdr_after = sep_eval.full_eval(out_mvdr, gt, avg=True)

# MVDR - B
out_mvdr_b = beamformers.MVDR(mix, spk, nn, frame_len=frame_len, frame_step=frame_step)
mvdr_after_b = sep_eval.full_eval(out_mvdr_b, gt2, avg=True)

# table
table = [["SDR",eval_before['sdr'], mvdr_after['sdr'], mvdr_after_b['sdr']],
         ["PESQ",eval_before['pesq'], mvdr_after['pesq'], mvdr_after_b['pesq']], 
         ["STOI",eval_before['stoi'], mvdr_after['stoi'], mvdr_after_b['stoi']]]

head = ["Measure", "Before", "MVDR", "MVDR B"]
print tabulate(table, headers=head, tablefmt='fancy_grid')

ipd.display(ipd.Audio(mix[0], rate=fs))
ipd.display(ipd.Audio(out_mvdr, rate=fs))
ipd.display(ipd.Audio(out_mvdr_b, rate=fs))
ipd.display(ipd.Audio(gt, rate=fs))
ipd.display(ipd.Audio(gt2, rate=fs))

╒═══════════╤══════════╤══════════╤══════════╕
│ Measure   │   Before │     MVDR │   MVDR B │
╞═══════════╪══════════╪══════════╪══════════╡
│ SDR       │ 2.82732  │ 4.36235  │ 2.92894  │
├───────────┼──────────┼──────────┼──────────┤
│ PESQ      │ 2.202    │ 2.619    │ 2.648    │
├───────────┼──────────┼──────────┼──────────┤
│ STOI      │ 0.660943 │ 0.801043 │ 0.689232 │
╘═══════════╧══════════╧══════════╧══════════╛


In [99]:
import itertools
a = np.random.rand(513, 126, 4, 4) + 1j * np.random.rand(513, 126, 4, 4)
b = np.random.rand(4, 513, 126) + 1j * np.random.rand(4, 513, 126)
k = np.einsum('abdc,cab->abd', a, b)
C = 4
G = np.zeros_like(a)

Yj = 0
for i in range(C):
    Yj += a[..., i] * b[i, ..., None]
    
np.mean(Yj - k)

0j

In [112]:
a = np.random.rand(4, 513, 126) + 1j * np.random.rand(4, 513, 126)
b = np.random.rand(4, 513, 126) + 1j * np.random.rand(4, 513, 126)
C = 4
Rjj = np.zeros((513, 126, 4, 4), dtype='complex')
for (i1, i2) in itertools.product(range(C), range(C)):
    Rjj[..., i1, i2] = a[i1, ...] * np.conj(b[i2, ...])

k = np.einsum('abc,dbc->bcad', a, np.conj(b))

np.mean(Rjj - k)

0j

In [131]:
eps = 1e-15
def invert2(M):
    _r = []
    for k in M:
        _r.append(np.linalg.inv(k + np.eye(4, dtype='complex') * eps))
    return np.array(_r)

def fast_inverse2(A):
    identity = np.identity(A.shape[2], dtype=A.dtype)
    return np.array([np.linalg.solve(x, identity) for x in A])



In [3]:
print "a"

a


In [132]:
a = np.random.rand(512, 4, 4) + 1j * np.random.rand(512, 4, 4)

A1 = invert(a)
A2 = fast_inverse2(a)

print np.mean(A1 - A2)

(0.00013325231807559184+0.0007620005221923847j)


In [ ]:
# Oracle MVDR
# Oracle MWF
# Beamformit 
# SDW_MWF
# MWF
import soundfile as sf
reload(beamformers)
reload(sep_eval)
fs = 8000
receptive_field = 0.128  # in s
frame_len = int(fs * receptive_field)
frame_step = int(frame_len / 4)
n_ch = 4


spk, fs = sf.read('wavs/spk.wav')
spk = spk.T
# spk /= np.max(np.abs(spk))

# talker
nn, fs = sf.read('wavs/nn.wav')
nn = nn.T
# nn /= np.max(np.abs(nn))

# noise
# nn = ex[2][:n_ch, 0]
# nn /= np.max(np.abs(nn))

gt, fs = sf.read('wavs/gt.wav')

mix, fs = sf.read('wavs/mix.wav')
mix = mix.T
# mix /= np.max(np.abs(mix))

eval_before = sep_eval.full_eval(mix[0], gt, average=True) 

# MVDR
print("TD_MWF - With reference")
out_tdmvdr_a = beamformers.TD_MWF(mix, nn, spk, frame_len=frame_len)
tdmvdr_after_a = sep_eval.full_eval(out_tdmvdr_a, gt, average=True)

# table
table = [["SDR",eval_before['sdr'], tdmvdr_after_a['sdr']],
         ["PESQ",eval_before['pesq'], tdmvdr_after_a['pesq']], 
         ["STOI",eval_before['stoi'], tdmvdr_after_a['stoi']]]

head = ["Measure", "Before", "MSNR"]
print tabulate(table, headers=head, tablefmt='fancy_grid')

sf.write('wavs/td_mwf_wref.wav', out_tdmvdr_a, 8000)


TD_MWF - With reference


In [4]:
sf.write('wavs/td_mvdr_wref.wav', out_tdmvdr_a, 8000)